# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [16]:
#%pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 12.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
import pandas as pd
import pyarrow as pa
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM tracks LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///chinook.sqlite
Done.


[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

 * sqlite:///chinook.sqlite
Done.


[(1, 'For Those About To Rock (We Salute You)', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 343719, 11170334, 0.99),
 (2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, 0.99),
 (3, 'Fast As a Shark', 3, 2, 1, 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619, 3990994, 0.99),
 (4, 'Restless and Wild', 3, 2, 1, 'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman', 252051, 4331779, 0.99),
 (5, 'Princess of the Dawn', 3, 2, 1, 'Deaffy & R.A. Smith-Diesel', 375418, 6290521, 0.99)]

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`

In [6]:
result = %sql SELECT t.Name as TrackName, a.Title as AlbumName \
        from  tracks t JOIN albums a USING(albumid) LIMIT 5;

display(result)

 * sqlite:///chinook.sqlite
Done.


[('For Those About To Rock (We Salute You)', 'For Those About To Rock We Salute You'),
 ('Put The Finger On You', 'For Those About To Rock We Salute You'),
 ("Let's Get It Up", 'For Those About To Rock We Salute You'),
 ('Inject The Venom', 'For Those About To Rock We Salute You'),
 ('Snowballed', 'For Those About To Rock We Salute You')]

2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`

In [18]:
result = %sql SELECT Country, COUNT(CustomerId) as CustomerCount \
FROM customers \
GROUP BY Country \
ORDER BY CustomerCount DESC;

# Display query result, note that Pandas DataFrame is returned!
display(result)

 * sqlite:///chinook.sqlite
Done.


[('USA', 13),
 ('Canada', 8),
 ('France', 5),
 ('Brazil', 5),
 ('Germany', 4),
 ('United Kingdom', 3),
 ('Portugal', 2),
 ('India', 2),
 ('Czech Republic', 2),
 ('Sweden', 1),
 ('Spain', 1),
 ('Poland', 1),
 ('Norway', 1),
 ('Netherlands', 1),
 ('Italy', 1),
 ('Ireland', 1),
 ('Hungary', 1),
 ('Finland', 1),
 ('Denmark', 1),
 ('Chile', 1),
 ('Belgium', 1),
 ('Austria', 1),
 ('Australia', 1),
 ('Argentina', 1)]

3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`


In [8]:
result = %sql SELECT g.Name as GenreName, COUNT(t.TrackId) as TrackCount \
FROM genres g \
JOIN tracks t USING(GenreId) \
GROUP BY GenreName \
ORDER BY TrackCount DESC \
LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)

 * sqlite:///chinook.sqlite
Done.


[('Rock', 1297),
 ('Latin', 579),
 ('Metal', 374),
 ('Alternative & Punk', 332),
 ('Jazz', 130)]

4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`


In [14]:
result = %sql SELECT c.FirstName, c.LastName, ROUND(AVG(i.Total),2) as AverageInvoiceTotal \
FROM customers c \
JOIN invoices i ON c.CustomerId = i.CustomerId \
GROUP BY c.CustomerId \
ORDER BY AverageInvoiceTotal DESC;

display(result)

 * sqlite:///chinook.sqlite
Done.


[('Helena', 'Holý', 7.09),
 ('Richard', 'Cunningham', 6.8),
 ('Luis', 'Rojas', 6.66),
 ('Ladislav', 'Kovács', 6.52),
 ('Hugh', "O'Reilly", 6.52),
 ('Frank', 'Ralston', 6.23),
 ('Julia', 'Barnett', 6.23),
 ('Fynn', 'Zimmermann', 6.23),
 ('Puja', 'Srivastava', 6.11),
 ('Astrid', 'Gruber', 6.09),
 ('Victor', 'Stevens', 6.09),
 ('Terhi', 'Hämäläinen', 5.95),
 ('František', 'Wichterlová', 5.8),
 ('Isabelle', 'Mercier', 5.8),
 ('Johannes', 'Van der Berg', 5.8),
 ('Luís', 'Gonçalves', 5.66),
 ('François', 'Tremblay', 5.66),
 ('Bjørn', 'Hansen', 5.66),
 ('Jack', 'Smith', 5.66),
 ('Dan', 'Miller', 5.66),
 ('Heather', 'Leacock', 5.66),
 ('João', 'Fernandes', 5.66),
 ('Wyatt', 'Girard', 5.66),
 ('Jennifer', 'Peterson', 5.52),
 ('Tim', 'Goyer', 5.52),
 ('Camille', 'Bernard', 5.52),
 ('Dominique', 'Lefebvre', 5.52),
 ('Joakim', 'Johansson', 5.52),
 ('Manoj', 'Pareek', 5.52),
 ('Leonie', 'Köhler', 5.37),
 ('Daan', 'Peeters', 5.37),
 ('Kara', 'Nielsen', 5.37),
 ('Eduardo', 'Martins', 5.37),
 ('Alexandre', 'Rocha', 5.37),
 ('Roberto', 'Almeida', 5.37),
 ('Fernanda', 'Ramos', 5.37),
 ('Mark', 'Philips', 5.37),
 ('Frank', 'Harris', 5.37),
 ('Michelle', 'Brooks', 5.37),
 ('Kathy', 'Chase', 5.37),
 ('John', 'Gordon', 5.37),
 ('Patrick', 'Gray', 5.37),
 ('Robert', 'Brown', 5.37),
 ('Edward', 'Francis', 5.37),
 ('Martha', 'Silk', 5.37),
 ('Aaron', 'Mitchell', 5.37),
 ('Ellie', 'Sullivan', 5.37),
 ('Madalena', 'Sampaio', 5.37),
 ('Hannah', 'Schneider', 5.37),
 ('Niklas', 'Schröder', 5.37),
 ('Marc', 'Dubois', 5.37),
 ('Lucas', 'Mancini', 5.37),
 ('Stanisław', 'Wójcik', 5.37),
 ('Enrique', 'Muñoz', 5.37),
 ('Emma', 'Jones', 5.37),
 ('Phil', 'Hughes', 5.37),
 ('Steve', 'Murray', 5.37),
 ('Mark', 'Taylor', 5.37),
 ('Diego', 'Gutiérrez', 5.37)]

5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [13]:
result = %sql SELECT c.FirstName, c.LastName, ROUND(SUM(i.Total),2) as TotalAmountSpent \
FROM customers c \
JOIN invoices i ON c.CustomerId = i.CustomerId \
GROUP BY c.CustomerId \
ORDER BY TotalAmountSpent DESC \
LIMIT 1;

display(result)


 * sqlite:///chinook.sqlite
Done.


[('Helena', 'Holý', 49.62)]

In [17]:
%%sql

-- Query 1: Select top 5 genres with the highest number of tracks
SELECT g.Name as GenreName, COUNT(t.TrackId) as TrackCount
FROM genres g
JOIN tracks t USING(GenreId)
GROUP BY GenreName
ORDER BY TrackCount DESC
LIMIT 5;

-- Query 2: Identify the customer who spent the most on music purchases
SELECT c.FirstName, c.LastName, SUM(i.Total) as TotalAmountSpent
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalAmountSpent DESC
LIMIT 1;


 * sqlite:///chinook.sqlite
Done.
Done.


[('Helena', 'Holý', 49.620000000000005)]